In [1]:
import json
import pandas as pd

with open('data/sharif.json') as f:
    data = json.load(f)

In [2]:
k = set()
for i in range(len(data['messages'])):
    for key in data['messages'][i].keys():
        k.add(key)
k = list(k)

In [3]:
df = pd.DataFrame.from_dict(data['messages'], orient='columns')[k]

In [4]:
from datetime import datetime, timedelta

df['date-time'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))

### Find top discussions

In [16]:
import math

offset = timedelta(seconds=3600)
starters = []
starters_count = {}
starters_time = {}

for i in range(len(df)):
    if len(starters) == 0:
        starters.append(i)
        starters_count[i] = 1
        starters_time[i] = df.iloc[i]['date-time']
    else:
        last_starter = starters[len(starters)-1]
        if not math.isnan(df.iloc[i]['reply_to_message_id']) or df.iloc[i]['date-time'] - starters_time[last_starter] <= offset:
            starters_count[last_starter] += 1
            starters_time[last_starter] = df.iloc[i]['date-time']
        else:
            starters.append(i)
            starters_count[i] = 1
            starters_time[i] = df.iloc[i]['date-time']

In [17]:
sorted_starters_count = sorted(starters_count.items(), key=lambda item: item[1], reverse=True)

In [27]:
top = [x[0] for x in sorted_starters_count[:20]]
df.iloc[top][['text', 'from', 'date-time']]

,text,from,date-time
33391,پاشید آماده شید واسه رای دادن دیگه :)),None,2017-05-19 06:46:03
29486,يه چيزي ميفرستم حجت رو تمام ميكنم :)),Ahmadreza Usefkhani,2017-05-17 09:07:56
42112,کسی میدونه آز تو تابستون چند جلسه است؟,سید امیرحسین حمیدیان,2017-06-06 11:42:07
31434,تندگویان,Sina Faraji,2017-05-18 08:42:15
19756,[✅ ابراهیم رییسی به دلیل قرار گرفتن نامش در لی...,naVd m,2017-04-28 11:40:47
202,انقدر ادمای غیر ce تو لابی زیاده,Iman Hosseini,2017-01-30 10:05:41
1540,"[{'type': 'hashtag', 'text': '#اقدام_دانشگاه_ص...",Mojtaba Fayazbakhsh,2017-01-31 14:54:00
25207,[«میرسلیم در برنامه رادیو معارف: همه‌ رشته‌های...,Mohammad Sadegh,2017-05-12 08:56:20
2667,"[مصاحبه تولسی گبرد عضو دموکرات , {'type': 'has...",Mojtaba Fayazbakhsh,2017-02-01 08:24:02
28463,"[{'type': 'link', 'text': 'https://instagram.c...",Ahmadreza Usefkhani,2017-05-16 09:08:15


In [28]:
for x in top:
    print(df.iloc[x]['date-time'] - df.iloc[x-1]['date-time'])

0 days 03:25:21
0 days 01:26:57
0 days 04:15:36
0 days 03:32:47
0 days 01:22:45
0 days 01:35:13
0 days 01:35:28
0 days 06:43:40
0 days 05:34:20
0 days 03:00:28
0 days 01:25:00
0 days 02:39:12
0 days 01:26:26
0 days 01:46:39
0 days 04:06:30
0 days 02:35:36
0 days 01:17:22
0 days 01:31:57
0 days 01:27:33
0 days 01:34:06


### Find close people

In [17]:
def update_2dict(d, k1, k2, v):
    if k1 not in d:
        d[k1] = {}
    if k2 not in d[k1]:
        d[k1][k2] = v
    else:
        d[k1][k2] += v

In [30]:
reply_ids = []
date_times = []
froms = []
ids = []

for i in range(len(df)):
    x = df.iloc[i]
    ids.append(x['id'])
    reply_ids.append(x['reply_to_message_id'])
    date_times.append(x['date-time'])
    froms.append(x['from'])

In [33]:
FORWARD, TIME, MESSAGE = 100.0, 10.0, 20.0
score = {}

for i in range(len(df)):
    for j in range(i+1, min(i+100, len(df))):
        s = FORWARD if reply_ids[j] == ids[i] else 0.0
        s += TIME / ((date_times[j] - date_times[i]).total_seconds() + 1.0)
        s += MESSAGE / (j-i)
        update_2dict(score, froms[i], froms[j], s)

In [57]:
import math

flat_scores = {}
messages_count = df.groupby(['from']).count()['id']

for k1, v1 in score.items():
    for k2, v2 in v1.items():
        if k1 != k2 and str(k1) != 'nan' and str(k1) != 'None' and str(k2) != 'nan' and str(k2) != 'None':
            flat_scores[str(k1) + '-' + str(k2)] = v2 / math.sqrt(messages_count[k1] * messages_count[k2])

In [58]:
import operator
sorted_scores = sorted(flat_scores.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
sorted_scores[:50]